## PEGASUS Local

In [7]:
import sys
sys.path.append('..')

import torch
from parser.parser import Parser
from Scorer import Score
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'google/pegasus-large'
preTrainedModel = '../preTrainedModel/'
torch_device = 'cpu'

### Initializing the tokenizer and the model

In [8]:
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(preTrainedModel).to(torch_device)

### Function For Getting response from the model

In [9]:
def get_response(input_text):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=1024, return_tensors="pt").to(torch_device)
  gen_out = model.generate(**batch,max_length=128,num_beams=5, num_return_sequences=1, do_sample = True, temperature=1.5)
  output_text = tokenizer.batch_decode(gen_out, skip_special_tokens=True)
  return output_text

### Preparing Dataset

In [10]:
from Datasets.DataLoader import DataLoader

#Loading the datasets through the custom dataloader.
datasetLoader = DataLoader(datasetName='arxiv')
arxiv_test = datasetLoader.getData('../Datasets/', split='test')
# datasetLoader.datasetName = 'pubmed'
# pubmed_test = datasetLoader.getData('../Datasets/', split='test')

## Running a sample instance

In [5]:
content = '.'.join(arxiv_test.loc[1, 'article_text'])
pegasusSummary = get_response(content)[0] #Getting the Response from the model
print("Sample Summary : ")
print(pegasusSummary)

goldenSummary = '.'.join(arxiv_test.loc[1, 'abstract_text'])
score = Score(trueSummary = goldenSummary, predSummary = pegasusSummary)
print("Sample Score : ")
for key, value in score.rougeScore().items():
    print(f"Criteria:{key}, Score:{value}" )

Sample Summary : 
in this paper, we consider the direct detection of gravitational waves ( gws ) by pulsar timing arrays ( ptas ).
Sample Score : 
Criteria:rouge1, Score:Score(precision=0.5882352941176471, recall=0.10101010101010101, fmeasure=0.17241379310344826)
Criteria:rouge2, Score:Score(precision=0.25, recall=0.04081632653061224, fmeasure=0.07017543859649122)
Criteria:rougeL, Score:Score(precision=0.5294117647058824, recall=0.09090909090909091, fmeasure=0.15517241379310345)


In [6]:
from evaluate import load
bertscore = load("bertscore")
results = bertscore.compute(predictions=[pegasusSummary], references=[goldenSummary], lang="en")
print(results['f1'][0]*100)

2023-11-20 22:15:07.796913: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 22:15:07.796961: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 22:15:07.815075: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 22:15:08.806721: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task t

83.75772833824158


In [ ]:
import time
import numpy as np

# pick only the first 1000 rows from the dataframes
arxiv_test = arxiv_test[:10]
# pubmed_test = pubmed_test[:100]

# creating 'Gold Summary' column
def mapping(row):
    row['Gold Summary'] = ''.join(row['abstract_text'])
    return row

arxiv_test = arxiv_test.apply(mapping, axis=1)
# pubmed_test = pubmed_test.apply(mapping, axis=1)

# generating summaries
def generateSummary(row):
    article = ''.join(row['article_text'])
    summary = get_response(article)[0]
    row['Generated Summary'] = summary
    print(f"Generated summary for {row['article_id']}.")
    return row

In [ ]:
from Evaluation.evaluation import rougeScores

start_time = time.time()
arxiv_test = arxiv_test.apply(generateSummary, axis=1)
print('Time taken for arxiv: ', time.time() - start_time)

arxiv_test, rougeScoresArxiv = rougeScores(arxiv_test)
# pubmed_test, rougeScoresPubmed = rougeScores(pubmed_test)

# printing the results
print('arxiv')
print('rouge1: ', np.mean([ score.fmeasure for score in rougeScoresArxiv['rouge1'] ]))
# print('rouge2: ', np.mean([ score.fmeasure for score in rougeScoresArxiv['rouge2'] ]))
# print('rougeL: ', np.mean([ score.fmeasure for score in rougeScoresArxiv['rougeL'] ]))
print('BertScore : ', np.mean( [score for score in rougeScoresArxiv['bertScore']]))
# saving the results
arxiv_test.to_csv('arxiv_test_pegasus.csv')
# pubmed_test.to_csv('pubmed_test_pegasus.csv')